# Assignment

How can we use Bayesian models to compare two distributions? It turns out that we can compare Bayesian models in several ways. In this assignment, we will compute and compare credible intervals of the posterior distribution of a model parameter.

Before we begin, let's load the libraries and functions we need.

In [1]:
%%capture

import pandas as pd
import numpy as np
import scipy
import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as plco

In [2]:
def posterior(prior, like):
    post = prior * like # compute the product of the probabilities
    return post / post.sum() # normalize the distribution to sum to unity

num_samples = 100000
lower_q, upper_q = [.025, .975]

def plot_ci(p, post, num_samples, lower_q, upper_q, fig=None, name='Posterior Density'):
    ## This function computes a credible interval using an assumption
    ## of symmetry in the bulk of the distribution to keep the 
    ## calculation simple. 
    ## Compute a large sample by resampling with replacement
    samples = np.random.choice(p, size = num_samples, replace = True, p = post)
    ci = np.percentile(samples, [lower_q*100, upper_q*100]) # compute the quantiles
    
    interval = upper_q - lower_q
    colors = plco.qualitative.Plotly
    
    if fig:
        color = [colors[2], colors[3]]
        text = f'{fig.layout.annotations[0].text} | {name.split(' ')[0]} {interval:.3f} credible interval is {ci[0]:.3f} to {ci[1]:.3f}'
    
    if fig is None:
        fig = go.Figure()
        color = [colors[0], colors[1]]
        text = f'{name.split(' ')[0]} {interval:.3f} credible interval is {ci[0]:.3f} to {ci[1]:.3f}'
    
    # Add posterior density line
    fig.add_trace(go.Scatter(x=p, y=post, mode='lines', line=dict(color=color[0]), name=name))

    # Add credible interval lines
    fig.add_trace(go.Scatter(x=[ci[0], ci[0]], y=[0, max(post)], mode='lines', line=dict(color=color[1]), name='Lower CI'))
    fig.add_trace(go.Scatter(x=[ci[1], ci[1]], y=[0, max(post)], mode='lines', line=dict(color=color[1]), name='Upper CI'))

    # Update layout
    fig.update_layout(
        title=f'Posterior density with {interval:.3f} credible interval',
        xaxis_title='Parameter value',
        yaxis_title='Density',
        showlegend=True,
    )
    fig.add_annotation(
        x=0,
        y=1.14,
        showarrow=False,
        text=text,
        xref='paper',
        yref='paper'
    )

    return fig


In [3]:
def plot_pp(x, p, l, pp):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=l, mode='lines', line=dict(color='lightgrey', width=10), name='likelihood'))
    fig.add_trace(go.Scatter(x=x, y=p, mode='lines', line=dict(color='blue', width=2), name='prior'))
    fig.add_trace(go.Scatter(x=x, y=pp, mode='lines', line=dict(color='red', width=2), name='posterior'))

    # Update layout
    fig.update_layout(
        title='Prior, likelihood and posterior distributions',
        xaxis_title='p',
        yaxis_title='PDF of p',
        legend_title='Distributions'
    )

    fig.show()

For this example, we will compare the posterior distribution of the heights of sons to the heights of the mothers in the Galton family dataset. As a first step, we will compute and evaluate Bayesian models for the mean heights using a subset of just 25 observations. 

In [4]:
families = pd.read_csv('../../data/GaltonFamilies.csv', index_col = 0)
families.head()

,father,mother,midparentHeight,children,childNum,gender,childHeight
family,,,,,,,
001,78.5,67.0,75.43,4,1,male,73.2
001,78.5,67.0,75.43,4,2,female,69.2
001,78.5,67.0,75.43,4,3,female,69.0
001,78.5,67.0,75.43,4,4,female,69.0
002,75.5,66.5,73.66,4,1,male,73.5


- Plot the distributions of the height of mothers and sons for a sample of size 25. Do the distributions appear to be significantly different? <span style="color:red" float:right>[5 point]</span>

In [5]:
num_samples = 25

male_children = families[families['gender'] == 'male']

sample = male_children.sample(n=num_samples)

fig = px.histogram(sample, x=['childHeight','mother'], nbins=30, title="Distribution of Heights for Mother's & Male Children")
fig.update_layout(
    xaxis_title='Height',
    yaxis_title='Frequency',
    barmode='overlay',
    width=800,
    height=500
)
fig.update_traces(opacity=0.75,marker=dict(line_width=0.25, line_color="black"))
fig.show()

In [6]:
# Create a histogram using Plotly
fig = px.histogram(male_children, x='childHeight', nbins=30, title="Distribution of Heights for Male Children Population")
fig.update_layout(
    xaxis_title='Height',
    yaxis_title='Frequency',
    width=800,
    height=500
)
fig.update_traces(opacity=0.75, marker=dict(line_width=0.25, line_color="black"))
fig.show()

To perform this analysis, we need to select a prior distribution, which is easy, and to compute the likelihood, which is not as easy. So first, let's see how we can get the likelihood.

For this data, we will use a normal likelihood. For something like heights, using the normal distribution makes sense. For any individual sample $X$ if it follows the normal distribution with mean $\mu$ and standard deviation $\sigma$, then it probability density function is given by

$$X \sim N(\mu, \sigma) \Rightarrow P(X | \mu, \sigma) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp \Bigg[ -\frac{1}{2}\big(\frac{X - \mu}{\sigma}\big)^2\Bigg]$$

For a sample ${X_1, X_2, \cdots, X_n}$ of $n$ independent normally distributed observations has the following likelihood:

$$P(X_1, X_2, \cdots, X_n | \mu, \sigma) = \prod_{i = 1}^n P(X_i | \mu, \sigma) = \prod_{i=1}^n \frac{1}{\sqrt{2 \pi \sigma^2}} \exp \Bigg[ -\frac{1}{2}\big(\frac{X_i - \mu}{\sigma}\big)^2\Bigg]$$

Notice that we multiply the individual the probabilities to get the likelihood. We can do this because we assume that the data points are independent. This seems like a relatively safe assumption. 

Now let's simplify the above equation. We can simplify things by ignoring the terms outside the exponential, since they don't depend on the sample, and replacing $=$ (is equal to) with $\propto$ (is proportional to):

$$P(X_1, X_2, \cdots, X_n | \mu, \sigma) \propto \prod_{i = 1}^n \exp \Bigg[ -\frac{1}{2}\big(\frac{X_i - \mu}{\sigma}\big)^2\Bigg]$$

Moreover, we can rely on the property $e^a e^b = e^{a + b}$ to rewrite the right-hand side as

$$P(X_1, X_2, \cdots, X_n | \mu, \sigma) \propto \exp \Bigg[ -\frac{1}{2} \sum_{i = 1}^n \big(\frac{X_i - \mu}{\sigma}\big)^2\Bigg]$$

Finally, letting $\bar X = \frac{1}{n}\sum_{i=1}^n X_i$ we can rewrite $X_i - \mu$ as $X_i - \bar X + \bar X - \mu$ and after some rearranging we get this:

$$P(X_1, X_2, \cdots, X_n | \mu, \sigma) \propto \exp \Bigg[ -\frac{1}{2 \sigma^2}  \Bigg( \sum_{i = 1}^n (X_i - \bar{X})^2 + n(\bar{X} - \mu)^2 \Bigg) \Bigg]$$

To simplify the computations here, we will only estimate the posterior distribution of $\mu$. We will use a fixed empirical estimate of the standard deviation. A more complete analysis will also estimate the posterior distribution of $\sigma$.

- Now your task is to write a function called `likelihood` that computes the likelihood as given by the above equation. The input of the function is `mu` (the parameter) and `x` (the data) and its output is the likelihood of the data given the parameter. You may find it helpful to use `np.exp` and `np.sum`. <span style="color:red" float:right>[10 point]</span>

In [7]:
def likelihood(data, mu):
    n = len(data)
    X_bar = np.mean(data)
    sigma = np.std(data)
    ext_const = -0.5 / sigma ** 2
    # print(ext_const)
    sum_squared_deviations = np.sum((data - X_bar) ** 2)
    # print(sum_squared_deviations)
    int_const = n * (X_bar - mu) ** 2
    # print(int_const)
    inter = ext_const * (sum_squared_deviations + int_const)
    # print(inter)
    exponent = np.exp(inter)
    # print(exponent)
    exponent = exponent / np.sum(exponent)
    
    return exponent

def _normalize(data):
    return data / data.sum()

Now for the prior distribution, we will use a normal distribution centered at 70 with standard deviation 1. The code for the prior is already written. Examine it and make sure you understand each step.

In [8]:
N=1000
mu = np.linspace(60, 80, num = N)
prior_center = 70
prior = scipy.stats.norm.pdf(mu, loc = prior_center, scale = 1)
prior = prior / prior.sum() # normalize

Now we're ready to compute the posterior for both mother and son.

- Compute the posterior distribution for the sons (the column name is `childHeight`). You will need to compute the likelihood first. Then plot the prior, posterior and likelihood just like we did in class. <span style="color:red" float:right>[10 point]</span>

In [9]:
# Compute the likelihood
son_likelihood = likelihood(sample['childHeight'], mu)

# Compute the posterior
son_posterior = posterior(prior, son_likelihood)

# Plot the prior, likelihood, and posterior
plot_pp(mu, prior, son_likelihood, son_posterior)

- Compute the posterior distribution for mothers (the column name is `mother`). You will need to compute the likelihood first. Sons and mothers both use the same proir. Then plot the prior, posterior and likelihood just like we did in class. <span style="color:red" float:right>[5 point]</span>

In [10]:
# Compute the likelihood
mom_likelihood = likelihood(sample['mother'], mu)

# Compute the posterior
mom_posterior = posterior(prior, mom_likelihood)

# Plot the prior, likelihood, and posterior
plot_pp(mu, prior, mom_likelihood, mom_posterior)

To compare the posterior distributions of the mean heights of the sons to the distribution of the mean heights of the mothers, we compute and compare the confidence intervals. 

- Use the `plot_ci` function to plot credible intervals for mother and son heights. You can call the function twice in the same cell and the two distributions will be plotted next to each other in one plot. What conclusion do you draw about the heights of mothers versus sons? <span style="color:red" float:right>[10 point]</span>

In [11]:
num_samples = 100000
lower_q, upper_q = [.025, .975]

f = plot_ci(mu, son_posterior, num_samples, lower_q, upper_q, name="Son's Height Posterior Density")
f = plot_ci(mu, mom_posterior, num_samples, lower_q, upper_q, fig=f, name="Mother's Height Posterior Density")
f.show()

Since the CIs don't even overlap, it can follow that the sons were significantly taller than their mothers.

# End of assignment